## Problemas

1. Jupyter está ejecutando el notebook con el Python base (/opt/conda/bin/python) → por eso el assert falla.
2. En terminal estamos como root y además no tenemos conda inicializado en ese shell → por eso conda activate no existe.

**`sys.executable` te sale `/opt/conda/bin/python` aunque exista el env**
Las causas típicas:
- El kernel `inesagent_gpu` no está registrado en Jupyter (falta `ipykernel/kernelspec`).
- El kernelspec apunta a un python que ya no existe (te pasó antes con `/opt/conda/envs/.../bin/python`).
- Estás entrando en un servidor donde el env existe en disco pero Jupyter no lo ve (porque Jupyter corre con otro usuario / otra config).

**`conda activate` te falla: estás en un shell sin inicializar (y además en root)**
El error:
`CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.`

**Solución recomendada (robusta): arreglar kernel desde el notebook usando el Python del env**

Como ahora mismo estás ejecutando en `/opt/conda/bin/python`, no puedes usar conda activate cómodamente. Pero sí puedes:
- comprobar si el python del env existe
- registrar el kernelspec apuntando a ese python

In [2]:
#Paso A
from pathlib import Path
import os, sys, subprocess, textwrap

ENV_PY = Path("/home/jovyan/.conda/envs/inesagent_gpu/bin/python")
print("Current PY:", sys.executable)
print("Env python exists:", ENV_PY.exists(), ENV_PY)

# sanity: ejecutar el python del env
if ENV_PY.exists():
    out = subprocess.check_output([str(ENV_PY), "-c", "import sys; print(sys.executable)"]).decode()
    print("Env says:", out.strip())


Current PY: /opt/conda/bin/python3.11
Env python exists: False /home/jovyan/.conda/envs/inesagent_gpu/bin/python


In [3]:
#Paso B: instalar ipykernel dentro del env y registrar kernel
import subprocess
from pathlib import Path

ENV_PY = Path("/home/jovyan/.conda/envs/inesagent_gpu/bin/python")
assert ENV_PY.exists(), f"No existe: {ENV_PY}"

# 1) asegurar ipykernel en el env
subprocess.check_call([str(ENV_PY), "-m", "pip", "install", "-U", "ipykernel"])

# 2) registrar kernelspec (nombre visible en Jupyter)
subprocess.check_call([
    str(ENV_PY), "-m", "ipykernel", "install",
    "--user",
    "--name", "inesagent_gpu",
    "--display-name", "inesagent_gpu (py311)"
])

print("✅ Kernel registrado. Ahora: Kernel -> Change kernel -> inesagent_gpu (py311)")


AssertionError: No existe: /home/jovyan/.conda/envs/inesagent_gpu/bin/python

## Env python exists: False

`Env python exists: False /home/jovyan/.conda/envs/inesagent_gpu/bin/python`, entonces ese entorno NO está en disco ahora mismo (o no está en esa ruta). Por eso Jupyter cae al Python base /opt/conda/bin/python3.11.

La pregunta clave ya no es “cómo activar”, sino:

✅ ¿Dónde están quedando los entornos?
✅ ¿Se están borrando al hacer Stop server? (muy probable si se crean en /opt/conda/envs u otra zona efímera) 

**Por qué se te borra “todo” al reiniciar**

En muchos JupyterHub/servidores gestionados:
- /home/jovyan (NFS) = persistente ✅
- /opt/conda/envs o disco local del contenedor = efímero ❌

Tu proyecto (`/home/jovyan/inesagent`) persiste, por eso ROOT existe.
Pero si el env estaba en `/opt/conda/envs`, se lo lleva el reinicio.

**Recrear el env en ruta persistente (sin depender de `conda activate`)**

Vamos a crearlo explícitamente en:
`/home/jovyan/.conda/envs/inesagent_gpu`